# Causal Inference in Observational Studies

So far, we've looked at ways of measuring association in any dataset. We've also looked at how we can draw conclusions about causality in randomized controlled trials. But in many cases, randomized controlled trials are not viable. Some reasons for that might be:

* **Ethical considerations**: suppose we believe that a certain treatment might harm people. In order to test this belief in a randomized controlled trial, we would have to subject people to a treatment that we believe would harm them, which is profoundly unethical. In such cases, we need alternative ways of drawing causal conclusions: we have an obligation as data scientists and researchers to avoid making [the mistakes of the past](https://en.wikipedia.org/wiki/Unethical_human_experimentation_in_the_United_States).
* **Cost**: Randomized controlled trials can be expensive to run: we must find randomly chosen individuals that are representative the population; apply a treatment to half of them; and measure the outcomes, potentially over a long range of time. In some cases, this may be infeasible.
* **Sample size**: due to cost considerations, randomized controlled trials may not have enough subjects to draw strong statistical conclusions.

There is a wealth of data available on observational studies. In this section, we'll look at ways we can use this data to draw causal conclusions. As we saw when looking at association, the most common problem when trying to draw conclusions about causality is the presence of confounding variables. We'll focus our attention on methods that try to account for these confounding variables.

## Why it's hard

When dealing with data from observational studies, we can't just take the simple difference in means anymore. To see why not, let's return to our restaurants example. We'll do the same computations we did before, but this time from the potential outcomes perspective.

Consider the science table version of the data (where we've converted 👍 to 1 and 👎 to 0, so that averages correspond to percentages liked):

In [1]:
import pandas as pd

food = pd.read_csv('restaurants_counterfactuals.csv')
food.head()

,Dish rating for A,Dish rating for B,Year
0,1.0,NaN,2019
1,NaN,1.0,2019
2,1.0,NaN,2020
3,0.0,NaN,2020
4,0.0,NaN,2020


The `NaN` values indicate missing counterfactual outcomes that we don't get to observe: for example, dish 0 was ordered from Restaurant A, so we don't know what rating the critics would have given a dish from Restaurant B on that day.

Displaying the data this way requires us to think about it slightly differently, and highlights the implications of the potential outcomes framework! In particular:
* $Z_i$ corresponds to whether the critic ordered dish $i$ from restaurant A or B.
* $Y_i(B)$ and $Y_i(A)$ represent what-ifs: *if* the critics had ordered dish $i$ from a particular restaurant, would they have liked it?
* The year is our confounder (or "confounding variable"): we'll use $X_i$ to denote the year that dish $i$ was ordered in.

Our representation of the data this way makes it clear that we can't compute the ATE:

In [2]:
food_ATE = (food["Dish rating for B"] - food["Dish rating for A"]).mean()
food_ATE

nan

In trying to understand a causal relationship between choice of restaurant and liking the food, naively using the simple difference in observed means (SDO) can mislead us, as we already saw:

In [3]:
food_sdo = food["Dish rating for B"].mean() - food["Dish rating for A"].mean()
food_sdo

0.20000000000000007

## Confounding variables and the superpopulation model

In the case of a randomized controlled trial, we saw that we could use the difference in means estimator because of the independence between the treatment assignment $Z_i$ and the potential outcomes $\big(Y_i(0), Y_i(1)\big)$. But, in the case of an observational study, this independence assumption no longer holds. We've seen a few examples of this in the last few sections, where confounding variables affect both the treatment and the outcome.

In this section, we'll focus on a few methods that answer the question: if we also observe the confounding variable, can we determine whether the treatment has an effect on the outcome?

Our notation will be similar to what we've used before, and we'll add a new variable $X_i$ for each unit that represents the confounding information. We have tuples $(Z_i, Y_i(0), Y_i(1), X_i)$ for each unit that are i.i.d. (that is, the tuple for one unit is independent from the tuple for the next unit). This is called the **superpopulation model**. 

## Conditional average treatment effect and unconfoundedness

We'll define a new quantity called the **conditional average treatment effect (CATE)**. This is like the ATE we saw before, but conditioned on a particular value of $X$:

$$
\text{CATE} = \tau(x) = E[Y(1) - Y(0) | X = x]
$$

Introducing the conditioning alone doesn't help us: if we were to expand this using the tower property to condition on $Z$, we'll still have counterfactual terms. So why did we do it? It turns out that we only need to make one assumption for this to help us. This assumption is called **unconfoundedness**:

$$
    \big(Y(0), Y(1)\big) \perp \!\!\! \perp Z \mid X
$$

This is a statement of conditional independence. The potential outcomes $Y(0)$ and $Y(1)$ might not be independent of the treatment decision $Z$, but this assumption states that they are conditionally independent *if* we know the value of a confounding variable $X$. 

In the restaurant example above, unconfoundedness would mean that within each year, the decision whether to order from A or B (treatment) is independent of the ratings that dishes from A and B would get (potential outcomes). It does *not* state that the restaurant is conditionally independent of the rating: this is a subtle but important distinction.

If we assume unconfoundedness, then we can safely estimate the CATE using the difference in conditional means:

$$
\begin{align}
    \tau(x) 
        &= E[Y(1) - Y(0) | X = x] &\\
        &= E[Y(1) | X = x] - E[Y(0) | X = x] & {\tiny\text{(linearity of (conditional) expectation)}}\\
        &= E[Y(1) | X = x, Z = 1] - E[Y(0) | X = x, Z = 0] & {\tiny\text{(unconfoundedness)}}
\end{align}
$$

### From CATE to ATE

In most cases, what we're really interested in is the ATE. How do we get from the CATE to the ATE? As we've done so many times before, we can use the law of iterated expectation (tower property):

$$
E[Y(1) - Y(0)] = E_X[E_Y[Y(1)-Y(0) | X]]
$$

In other words,

$$
\begin{align}
    ATE 
        &= E[\tau(X)] \\
        &= \sum_x \tau(x) P(X=x)
\end{align}
$$

(where we would use an integral instead of a sum if $X$ is continuous).

What does this look like in practice? Let's go back to the restaurant example. Instead of computing the difference in means for the entire dataset, we'll do it separately, once for 2020 and once for 2019:

In [4]:
food_2020 = food[food['Year'] == 2020]
food_2019 = food[food['Year'] == 2019]

food_sdo_2020 = food_2020["Dish rating for B"].mean() - food_2020["Dish rating for A"].mean()
food_sdo_2019 = food_2019["Dish rating for B"].mean() - food_2019["Dish rating for A"].mean()
print(food_sdo_2020, food_sdo_2019)

-0.05555555555555558 -0.125


These correspond to the CATEs $\tau(2020)$ and $\tau(2019)$ respectively. Just like earlier, we see that these values have the opposite sign from the SDO computed on the entire data: Restaurant A was better-liked in 2020 and 2019. But, because the critics ordered from Restaurant A more in 2020 (which had lower ratings overall), Restaurant A looks worse in the aggregate data.

How do we combine these to estimate the ATE? We need the marginal probabilities $P(2020)$ and $P(2019)$:

In [5]:
p_2020 = (food['Year'] == 2020).mean()
p_2019 = 1 - p_2020

est_food_ATE = food_sdo_2020 * p_2020 + food_sdo_2019 * p_2019
est_food_ATE

-0.07870370370370372

This is our first causal inference from observational data! To state it formally:

**If we assume that restaurant choice and the potential outcomes are independent given the year (unconfoundedness), then choosing Restaurant A causes the critics to like dishes about 8% more**.

Note that we can't just make a blanket statement about causality: this statement depends heavily on our use of the unconfoundedness assumption. This is true in general: in order to draw causal inferences outside of randomized controlled experiments, we need to make assumptions. When reporting your causal conclusions, it's always important to communicate the assumptions you made!

## Outcome Regression

*In progress*

## Inverse propensity weighting

*In progress*